## Install Required Packages

In [5]:
%%bash
pip install numpy
pip install scipy
pip install --upgrade keras

## Imports

In [6]:
import numpy as np
from scipy import io
import os
import pickle

# Preprocessing code provided by the Kording Lab
from Neural_Decoding.preprocessing_funcs import bin_spikes
from Neural_Decoding.preprocessing_funcs import bin_output

## User Inputs (Modify as Necessary)

In [7]:
mat_dir = "Directions_Formatted" # Name of directory storing the .mat structs
binned_dir = "Directions_Binned" # Name of directory that will store binned results

fd_dt = 0.1 # Size of time bins for feeding (in seconds)
dr_dt = 0.5 # Size of time bins for drinking (in seconds)

## Preprocessing

In [8]:
# Get all .mat files
parent_dir = os.path.dirname(os.path.abspath("__file__"))
data_dir = os.path.join(parent_dir, mat_dir)
files = next(os.walk(data_dir))[2]

# Create directory to store binned results
results_dir = os.path.join(parent_dir, binned_dir)
if not os.path.exists(results_dir):
    os.mkdir(results_dir)

for f in files:
    # Load .mat file
    data = io.loadmat(data_dir + "/" + f)

    # Get neural and direction data
    if "M1" in f:
        spike_times = data['m1s']
    else:
        spike_times = data['s1s']
    outputs = data['outputs']
    output_times = data['output_times']

    # Squeeze neural data
    spike_times = np.squeeze(spike_times)
    for i in range(spike_times.shape[0]):
        spike_times[i] = np.squeeze(spike_times[i])
    
    t_start = output_times[0] # Time to start extracting data - here we just start at the beginning
    t_end = output_times[-1] # Time to stop extracting data - here we just stop at the end
    df = 1 # Downsampling of output (to make binning go faster) - 1 means no downsampling

    # Binning
    if "Fd" in f:
        spikes_binned = bin_spikes(spike_times, fd_dt, t_start, t_end)
        directions_binned = bin_output(outputs, output_times, fd_dt, t_start, t_end, df)
    else:
        spikes_binned = bin_spikes(spike_times, dr_dt, t_start, t_end)
        directions_binned = bin_output(outputs, output_times, dr_dt, t_start, t_end, df)
    
    # Save results 
    res_filename = f.split(".")[0] + ".pickle"
    with open(os.path.join(results_dir, res_filename), "wb") as res:
        pickle.dump([spikes_binned, directions_binned], res)